In [ ]:
# Clean the folder
import os

folders = [
    "/content/allrulessample",
    "/content/Extracted_Output"
]

for folder in folders:
    if not os.path.exists(folder):
        print(f"⚠️ Folder not found: {folder}")
        continue

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                # print(f"🗑️ Deleted file: {file_path}")
        except Exception as e:
            print(f"❌ Error deleting {file_path}: {e}")


⚠️ Folder not found: /content/allrulessample
⚠️ Folder not found: /content/Extracted_Output


In [ ]:
# Extract the Full rule Json file from Application Zip mentioned in zip_path to extract_to
import zipfile
import os

# Path to your ZIP file
zip_path = "/content/other.zip"

# Folder where files will be extracted
extract_to = "/content/allrulessample"

# Ensure the target folder exists
os.makedirs(extract_to, exist_ok=True)

# Extract all files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"✅ Files extracted to: {extract_to}")


✅ Files extracted to: /content/allrulessample


In [ ]:
# Extract the context json schema file from context_rule_schema Zip
import zipfile
import os

# Path to your ZIP file
zip_path = "/content/context_rule_schema.zip"

# Folder where files will be extracted
extract_to = "/content/content-rule-schema"

# Ensure the target folder exists
os.makedirs(extract_to, exist_ok=True)

# Extract all files
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"✅ Files extracted to: {extract_to}")


✅ Files extracted to: /content/content-rule-schema


In [ ]:
# Create tables to store Rule details and rule reference details - One Time
import json
import os
import sqlite3

# Folder containing JSON files
folder_path = '/content/allrulessample'

# SQLite database file
db_file = 'rules_new.db'

# Connect to SQLite database (it will create the file if it doesn't exist)
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create tables with description column for top_rules
cursor.execute('''
CREATE TABLE IF NOT EXISTS top_rules (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    key TEXT,
    rule_type TEXT,
    rule_class TEXT,
    name TEXT,
    description TEXT,
    rulejson TEXT,
    businessdescription TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS rule_references (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    top_rule_key TEXT,
    key TEXT,
    rule_type TEXT,
    rule_class TEXT,
    name TEXT
)
''')

cursor.execute("""
CREATE TABLE IF NOT EXISTS class_inheritance (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    top_class TEXT,
    inherited_class TEXT,
    inheritance_type TEXT
)
""")

conn.commit()
conn.close()


All JSON files have been processed and stored in SQLite database with description column.


In [ ]:
# Add Records from JSON files
import json
import os
import sqlite3

# Folder containing JSON files
folder_path = '/content/allrulessample'

# SQLite database file
db_file = 'rules.db'

# Connect to SQLite database (it will create the file if it doesn't exist)
conn = sqlite3.connect(db_file)
cursor = conn.cursor()



# Process all JSON files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Extract top-level rule info with description
        top_rule_info = (
            data.get("pzInsKey", ""),
            data.get("pxObjClass", ""),
            data.get("pyClassName", ""),
            data.get("pyRuleName", ""),
            data.get("pyDescription", "")  # Add description field
        )

        # Insert top-level rule
        cursor.execute('''
        INSERT INTO top_rules (key, rule_type, rule_class, name, description)
        VALUES (?, ?, ?, ?, ?)
        ''', top_rule_info)

        top_rule_key = data.get("pzInsKey", "")

        # Extract embedded rule references
        for ref in data.get("pxRuleReferences", []):
            rule_info = (
                top_rule_key,
                ref.get("pxRuleObjClass", ""),
                ref.get("pxRuleClassName", ""),
                ref.get("pyRuleName", "")
            )

            # Insert embedded rule reference
            cursor.execute('''
            INSERT INTO rule_references (top_rule_key, rule_type, rule_class, name)
            VALUES (?, ?, ?, ?)
            ''', rule_info)

# Commit and close connection
conn.commit()
conn.close()

print("All JSON files have been processed and stored in SQLite database with description column.")


All JSON files have been processed and stored in SQLite database with description column.


In [ ]:
# Populate Class Inheritance Table
import json
import sqlite3

db_file = "rules.db"
input_file = "/content/class_inheritance_1.json"

conn = sqlite3.connect(db_file)
cursor = conn.cursor()



# Load JSON
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

for item in data.get("pxResults", []):
    top_class = item.get("pyRuleName", "")
    py_note = item.get("pyNote", "")

    try:
        note_json = json.loads(py_note)
        for r in note_json.get("pxResults", []):
            cursor.execute("""
                INSERT INTO class_inheritance (top_class, inherited_class, inheritance_type)
                VALUES (?, ?, ?)
            """, (
                top_class,
                r.get("pyClassName", ""),
                r.get("pyInheritanceType", "")
            ))
    except json.JSONDecodeError:
        print(f"⚠️ Could not parse pyNote for {top_class}")

conn.commit()
conn.close()
print("✅ class_inheritance table populated.")

✅ class_inheritance table populated.


In [ ]:
# Set functionality for Extracting RuleJSON column data for Top_rules table - This step defines the functionality
import os
import json

# ✅ Folders
# source_folder = "/content/Extract_Business_From_Pega/Sample_Complete_Rule_Json"
source_folder = "/content/allrulessample"
selector_folder = "/content/content-rule-schema"
output_folder = "/content/Extracted_Output"

os.makedirs(output_folder, exist_ok=True)


def is_blank(node):
    if node is None:
        return True
    if isinstance(node, str) and node.strip() == "":
        return True
    if isinstance(node, (list, dict)) and len(node) == 0:
        return True
    return False


def extract(source, selector):
    """Recursively extract fields from source JSON based on selector JSON, skipping blanks."""
    result = {}

    for raw_field, selector_value in selector.items():
        field = raw_field[1:] if raw_field.startswith('.') else raw_field
        source_value = source.get(field) if isinstance(source, dict) else None

        # Prefix match handling (e.g. Utility1, End52, Start1)
        if (source_value is None or is_blank(source_value)) and isinstance(selector_value, dict) and isinstance(source, dict):
            prefix_matches = {}
            for source_key, candidate in source.items():
                if source_key.startswith(field) and isinstance(candidate, dict):
                    extracted = extract(candidate, selector_value)
                    if extracted:
                        prefix_matches[source_key] = extracted
            if prefix_matches:
                result[field] = prefix_matches
            continue

        # Normal match
        if source_value is None or is_blank(source_value):
            continue

        # Nested objects
        if isinstance(selector_value, dict) and isinstance(source_value, dict):
            nested = extract(source_value, selector_value)
            if nested:
                result[field] = nested

        # Arrays
        elif isinstance(selector_value, list) and isinstance(source_value, list):
            result_array = []
            if len(selector_value) > 0:
                template = selector_value[0]
                for item in source_value:
                    if isinstance(item, dict) and isinstance(template, dict):
                        extracted = extract(item, template)
                        if extracted:
                            result_array.append(extracted)
            if result_array:
                result[field] = result_array

        # Copy simple values
        elif not isinstance(selector_value, (list, dict)):
            result[field] = source_value

    return result




In [ ]:
# Extract Rule JSON and put in Top_Rules table - This steps populate to the table
import re
import os
import json
import sqlite3

db_path = "rules.db"

# --- Connect to database ---
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for file_name in os.listdir(source_folder):
    if file_name.endswith(".json"):
        # Remove prefix "full_" if present and suffix ".json"
        base_name = file_name.replace("full_", "").replace(".json", "")

        # Full path to the file
        file_path = os.path.join(source_folder, file_name)

        # Step 2: Load the JSON file and extract pxObjClass
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Step 3: Extract the field safely
        px_obj_class = data.get("pxObjClass").lower()
        base_name = px_obj_class

        # print(f"pxObjClass: {px_obj_class}")

        # Extract main rule type and optional numeric suffix
        match = re.match(r"(.+?)(_\d+)?$", base_name)
        rule_type = match.group(1)           # e.g., "definition"
        suffix = match.group(2) or ""        # e.g., "_1" or "_2" (if present)

        # ✅ Extract context word properly:
        # If file_name is "full_definition_634.json" → context_word = "definition"
        parts = base_name.split("_")
        if len(parts) > 1:
            context_word = parts[0] if parts[0] != "full" else parts[1]
        else:
            context_word = base_name
        context_word = context_word.lower()

        # ✅ Build correct paths
        source_path = os.path.join(source_folder, file_name)
        selector_path = os.path.join(selector_folder, f"context_{context_word}.json")
        output_path = os.path.join(output_folder, f"extracted_{rule_type}{suffix}.json")

        if not os.path.exists(selector_path):
            print(f"⚠️  Skipping {file_name} — no matching selector found: {selector_path}")
            continue

        # print(f"🔹 Processing {file_name} → using {os.path.basename(selector_path)}")

        # Load JSONs
        with open(source_path, "r", encoding="utf-8") as src_file:
            source_json = json.load(src_file)
        with open(selector_path, "r", encoding="utf-8") as sel_file:
            selector_json = json.load(sel_file)

        # Extract
        result = extract(source_json, selector_json)

        # Save output
        with open(output_path, "w", encoding="utf-8") as out_file:
            json.dump(result, out_file, indent=4, ensure_ascii=False)

        # print(f"✅ Saved extracted file → {output_path}\n")

        # --- Extract rule key (pzInsKey) ---
        pzInsKey = source_json.get("pzInsKey", None)
        if not pzInsKey:
            print(f"⚠️ No pzInsKey found in {file_name}, skipping DB insert.\n")
            continue

        # --- Save result in rule_json_store ---
        cursor.execute("""
            UPDATE top_rules
            SET rulejson = ?
            WHERE key = ?
        """, (
            json.dumps(result, ensure_ascii=False, indent=2),
            pzInsKey
        ))
        conn.commit()

        # print(f"💾 Stored JSON for key: {pzInsKey} in rule_json_store\n")

print("🎉 All matching Full_*.json files processed!")





⚠️  Skipping full_soap_3247.json — no matching selector found: /content/content-rule-schema/context_rule-service-soap.json
🎉 All matching Full_*.json files processed!


In [ ]:
# Approach-2 -show hierarchical data json which may be sent to LLM for High level Business
import sqlite3
import json

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()

# -----------------------------
# Fetch child rules for a given parent key
# -----------------------------
def fetch_children(parent_key):
    query = """
    SELECT DISTINCT
        rr.top_rule_key,
        t.key,
        t.name,
        t.rule_class,
        t.rule_type,
        t.businessdescription
    FROM rule_references rr
    INNER JOIN top_rules t
        ON rr.name = t.name
        AND rr.rule_type = t.rule_type
    WHERE rr.top_rule_key = ?
    ORDER BY t.name;
    """
    cursor.execute(query, (parent_key,))
    return cursor.fetchall()

# -----------------------------
# Recursive hierarchy builder
# -----------------------------
def build_rule_hierarchy(parent_key, visited=None):
    if visited is None:
        visited = set()

    if parent_key in visited:
        return []  # Prevent infinite recursion
    visited.add(parent_key)

    children = fetch_children(parent_key)
    hierarchy = []

    for top_rule_key, key, name, rule_class, rule_type, businessdescription in children:
        rule_type_upper = rule_type.upper()
        if "FLOWACTION" in rule_type_upper:
            label = "FlowAction"
        elif "FLOW " in rule_type_upper:
            label = "Flow"
        elif "HTML-SECTION" in rule_type_upper:
            label = "Section"
        else:
            label = rule_type.split('-')[-1].title()

        # Skip technical rules
        if any(key.startswith(p) for p in (
            "RULE-OBJ-PROPERTY",
            "RULE-OBJ-CLASS",
            "RULE-OBJ-FIELDVALUE",
            "RULE-OBJ-REPORT-DEFINITION",
            "RULE-DECLARE-EXPRESSIONS",
        )):
            continue

        # Clean up description
        desc = (businessdescription or "No business description").strip()
        desc = " ".join(desc.split())  # remove extra whitespace
        if len(desc) > 400:
            desc = desc[:400] + "..."  # truncate overly long text

        # Recursive call for nested children
        calls = build_rule_hierarchy(key, visited)

        # Build the node as nested dict
        node = {
            label: {
                "Name": name,
                "Description": desc
            }
        }

        if calls:
            node[label]["Calls"] = calls

        hierarchy.append(node)

    return hierarchy

# -----------------------------
# Build hierarchy from top-level case types
# -----------------------------
cursor.execute("""
    SELECT DISTINCT top_rule_key
    FROM rule_references
    WHERE top_rule_key LIKE 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT'
""")
top_keys = [row[0] for row in cursor.fetchall()]

final_output = []

for key in top_keys:
    children = build_rule_hierarchy(key)
    case_type = {
        "CaseType": {
            "Name": key,
            "Calls": children
        }
    }
    final_output.append(case_type)

# -----------------------------
# Output JSON
# -----------------------------
json_output = json.dumps(final_output, indent=2, ensure_ascii=False)
print(json_output)

# Optionally save to file
with open("rule_hierarchy_with_desc.json", "w", encoding="utf-8") as f:
    f.write(json_output)

conn.close()


[
  {
    "CaseType": {
      "Name": "RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT",
      "Calls": [
        {
          "Flow": {
            "Name": "Cancellation",
            "Description": "This rule, named \"NewSample\", starts when a new case is created. It then moves to two utility activities, \"InvokeCancelCase\" and \"DeleteInstinstancesForCancelCase\". The \"InvokeCancelCase\" activity invokes a cancel case with a specific page class, while the \"DeleteInstinstancesForCancelCase\" activity deletes instances for a cancel case with a specific case ID. Finally, the rule ends.",
            "Calls": [
              {
                "Model": {
                  "Name": "RemoveProjectTeam",
                  "Description": "This rule cancels an initiative when its associated case is closed. It iterates through the project team members, setting their assignment keys to empty strings. The rule also removes the SLA end date. 

In [ ]:
# Optional - Zip output
import shutil
import os

# ✅ Paths
folder_to_zip = "/content/Extracted_Output"   # Folder you want to zip
output_zip = "/content/extracted_output_Part2.zip"  # Output zip file

# Create zip
shutil.make_archive(base_name=output_zip.replace(".zip",""), format='zip', root_dir=folder_to_zip)

print(f"✅ Folder zipped successfully: {output_zip}")


✅ Folder zipped successfully: /content/extracted_output_Part2.zip


In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules_new.db')
cursor = conn.cursor()

# 1️⃣ View all top-level rules
print("Top-level rules:")
cursor.execute("SELECT count(*) FROM top_rules")
top_rules = cursor.fetchall()
for row in top_rules:
    print(row)

print("\nEmbedded rule references:")
# 2️⃣ View all embedded rule references
cursor.execute("SELECT count(*) FROM rule_references")
rule_refs = cursor.fetchall()
for row in rule_refs:
    print(row)


# Close connection
conn.close()


Top-level rules:
(2652,)

Embedded rule references:
(3,)


In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()

# 1️⃣ View all top-level rules
print("Top-level rules:")
# cursor.execute("SELECT rule_type,count(*) FROM top_rules where businessdescription is null group by rule_type")
# cursor.execute("SELECT rule_type,count(*) FROM top_rules group by rule_type")
# cursor.execute("SELECT key, rule_type, rulejson FROM top_rules where businessdescription is null LIMIT 10")
cursor.execute("SELECT count(*) FROM top_rules where LENGTH(rulejson) < 100000")
# cursor.execute ("Update top_rules set businessdescription = 'do later' where key = 'RULE-OBJ-ACTIVITY UNILEVER-MDMSHARED-WORK PARSEEXCEL_DATACOLLECT #20210810T101747.293 GMT'")
# conn.commit()
top_rules = cursor.fetchall()
for row in top_rules:
    print(row)

print("\nEmbedded rule references:")
# 2️⃣ View all embedded rule references
cursor.execute("SELECT count(*) FROM rule_references")
rule_refs = cursor.fetchall()
for row in rule_refs:
    print(row)


# Close connection
conn.close()


Top-level rules:
(19368,)

Embedded rule references:
(498288,)


In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()


# 3️⃣ Example: Join top-level rules with embedded rules
print("\nTop rules with their embedded rules:")
cursor.execute("""
SELECT t.key AS top_key, t.name AS top_name, r.key AS ref_key, r.name AS ref_name, t.description as description
FROM top_rules t
LEFT JOIN rule_references r
ON t.key = r.top_rule_key
""")
joined_rows = cursor.fetchall()
for row in joined_rows:
    print(row)

# Close connection
conn.close()


In [ ]:
# Support - If data base is locked
import sqlite3, gc

# Try to close any lingering connection object in memory
try:
    conn.close()
except:
    pass

gc.collect()
print("🧹 All old connections closed and garbage collected.")



🧹 All old connections closed and garbage collected.


In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()


# 3️⃣ Example: Join top-level rules with embedded rules
print("\nTop rules with their embedded rules:")
cursor.execute("""
SELECT *
    FROM rule_references
    WHERE top_rule_key LIKE 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT%'
""")
joined_rows = cursor.fetchall()
for row in joined_rows:
    print(row)

# Close connection
conn.close()



Top rules with their embedded rules:
(10332, 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', None, 'Rule-Obj-Property', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint', 'FAReminder2')
(10333, 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', None, 'Rule-Obj-Property', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint', 'FASLAFinal')
(10334, 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', None, 'Rule-Obj-Property', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint-CCA', 'CommonCodeApproverEmail')
(10335, 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', None, 'Rule-Obj-Property', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint-CCA', 'DueCCApprovalDate')
(10336, 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', None,

In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()


# 3️⃣ Example: Join top-level rules with embedded rules
print("\nTop rules with their embedded rules:")
cursor.execute("""
SELECT *
    FROM top_rules
    WHERE key LIKE 'RULE-OBJ-FLOW UNILEVER-FW-MMDM-WORK-PRODUCTMANAGEMENT-MAINT MDMOPERATION #20210503T060903.349 GMT%'
""")
joined_rows = cursor.fetchall()
for row in joined_rows:
    print(row)

# Close connection
conn.close()



Top rules with their embedded rules:
(3749, 'RULE-OBJ-FLOW UNILEVER-FW-MMDM-WORK-PRODUCTMANAGEMENT-MAINT MDMOPERATION #20210503T060903.349 GMT', 'Rule-Obj-Flow', 'Unilever-FW-MMDM-Work-ProductManagement-Maint', 'MDMOperation', 'Cancellation Flow', '{\n  "pyCategory": "FlowStandard",\n  "pyFromTasks": [\n    {\n      "pxObjClass": "Embed-Rule-Obj-Flow-FromTasks",\n      "pyFromTaskName": "ASSIGNMENT63"\n    },\n    {\n      "pxObjClass": "Embed-Rule-Obj-Flow-FromTasks",\n      "pyFromTaskName": "Utility1"\n    },\n    {\n      "pxObjClass": "Embed-Rule-Obj-Flow-FromTasks",\n      "pyFromTaskName": "Start1"\n    }\n  ],\n  "pyModelProcess": {\n    "pyConnectors": {\n      "Transition": {\n        "Transition1": {\n          "pxSubscript": "Transition1",\n          "pxObjClass": "Data-MO-Connector-Transition",\n          "pyCategory": "FlowStandard",\n          "pyConditionType": "Always",\n          "pyConnectorType": "Transition",\n          "pyFrom": "Start1",\n          "pyLikelihood

In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()


# 3️⃣ Example: Join top-level rules with embedded rules
print("\nTop rules with their embedded rules:")
cursor.execute("""
select rule_type, count(*) from top_rules where businessdescription is null group by rule_type
""")
joined_rows = cursor.fetchall()
for row in joined_rows:
    print(row)

# Close connection
conn.close()



Top rules with their embedded rules:
('Rule-Declare-Pages', 664)
('Rule-HTML-Section', 166)
('Rule-Obj-CaseType', 1)
('Rule-Obj-Flow', 19)
('Rule-Obj-FlowAction', 75)
('Rule-Obj-Model', 935)
('Rule-Obj-Report-Definition', 493)
('Rule-Obj-When', 607)


In [ ]:
# DEBUG - Check the Table Records
import sqlite3

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()


# 3️⃣ Example: Join top-level rules with embedded rules
print("\nTop rules with their embedded rules:")
cursor.execute("""
SELECT DISTINCT
        rr.top_rule_key,
        t.key,
        t.name,
        t.rule_class,
        rr.rule_class,
        rr.rule_type,
        t.rule_type,
        t.businessdescription
    FROM rule_references rr
    INNER JOIN top_rules t
        ON rr.name = t.name
        AND rr.rule_type = t.rule_type
    WHERE rr.top_rule_key = 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT'
    ORDER BY t.name;
""")
joined_rows = cursor.fetchall()
for row in joined_rows:
    print(row)

# Close connection
conn.close()



Top rules with their embedded rules:
('RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', 'RULE-OBJ-FLOW UNILEVER-FW-MMDM-WORK-PRODUCTMANAGEMENT-MAINT CANCELLATION #20211025T082539.064 GMT', 'Cancellation', 'Unilever-FW-MMDM-Work-ProductManagement-Maint', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint', 'Rule-Obj-Flow', 'Rule-Obj-Flow', None)
('RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', 'RULE-OBJ-FLOW UNILEVER-FW-MMDM-WORK-PRODUCTMANAGEMENT-MAINT COUNTRYAPPROVAL #20220302T085210.024 GMT', 'CountryApproval', 'Unilever-FW-MMDM-Work-ProductManagement-Maint', 'Unilever-SEAA-MMDM-Work-ProductManagement-Maint', 'Rule-Obj-Flow', 'Rule-Obj-Flow', None)
('RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT', 'RULE-OBJ-FLOW UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT COUNTRYAPPROVAL #20230906T145911.019 GMT', 'CountryApproval', 'U

In [ ]:
# Support
import sqlite3

# Path to your SQLite database
db_file = "rules.db"

# Connect to database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# SQL query to count duplicates
query = """
SELECT
    top_rule_key,
    rule_type,
    rule_class,
    name,
    COUNT(*) AS duplicate_count
FROM rule_references
GROUP BY top_rule_key, rule_type, rule_class, name
HAVING COUNT(*) > 1
ORDER BY duplicate_count DESC;
"""

cursor.execute(query)
duplicates = cursor.fetchall()

# Print results
if duplicates:
    print("🔁 Duplicate rows found:\n")
    for row in duplicates:
        print(f"top_rule_key={row[0]}, rule_type={row[1]}, rule_class={row[2]}, name={row[3]}, count={row[4]}")
    print(f"\nTotal duplicate groups: {len(duplicates)}")
else:
    print("✅ No duplicate rows found.")

# Close connection
conn.close()


✅ No duplicate rows found.


In [ ]:
# Support
import sqlite3

# Path to your SQLite database
db_file = "rules.db"

# Connect to database
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

print("🔍 Finding and removing duplicate rows from rule_references...")

# Step 1: Delete duplicates while keeping the lowest ID
delete_query = """
DELETE FROM rule_references
WHERE id NOT IN (
    SELECT MIN(id)
    FROM rule_references
    GROUP BY top_rule_key, rule_type, rule_class, name
);
"""

cursor.execute(delete_query)
conn.commit()

# Step 2: Check how many rows remain
cursor.execute("SELECT COUNT(*) FROM rule_references;")
remaining = cursor.fetchone()[0]

print(f"✅ Duplicates removed successfully. Remaining rows in table: {remaining}")

conn.close()


🔍 Finding and removing duplicate rows from rule_references...
✅ Duplicates removed successfully. Remaining rows in table: 216946


In [ ]:
# Debug - Approach-2 -show hierarchical data json which may be sent to LLM for High level Business
import sqlite3
import json

# Connect to the database
conn = sqlite3.connect('rules.db')
cursor = conn.cursor()

# -----------------------------
# Fetch child rules for a given parent key
# -----------------------------
def fetch_children(parent_key):
    query = """
    SELECT DISTINCT
    rr.top_rule_key,
    t.key,
    t.name,
    t.rule_class,
    t.rule_type,
    t.businessdescription
FROM rule_references rr
INNER JOIN top_rules t
    ON rr.name = t.name
    AND rr.rule_type = t.rule_type
    AND (
        rr.rule_class = t.rule_class
        OR t.rule_class IN (
            SELECT inherited_class
            FROM class_inheritance
            WHERE top_class = rr.rule_class
        )
    )
WHERE rr.top_rule_key = ?
ORDER BY t.name;
    """
    cursor.execute(query, (parent_key,))
    return cursor.fetchall()

# -----------------------------
# Recursive hierarchy builder
# -----------------------------
def build_rule_hierarchy(parent_key, visited=None):
    if visited is None:
        visited = set()

    if parent_key in visited:
        return []  # Prevent infinite recursion
    visited.add(parent_key)

    children = fetch_children(parent_key)
    hierarchy = []

    for top_rule_key, key, name, rule_class, rule_type, businessdescription in children:
        rule_type_upper = rule_type.upper()
        if "FLOWACTION" in rule_type_upper:
            label = "FlowAction"
        elif "FLOW " in rule_type_upper:
            label = "Flow"
        elif "HTML-SECTION" in rule_type_upper:
            label = "Section"
        else:
            label = rule_type.split('-')[-1].title()

        # Skip technical rules
        if any(key.startswith(p) for p in (
            "RULE-OBJ-PROPERTY",
            "RULE-OBJ-CLASS",
            "RULE-OBJ-FIELDVALUE",
            "RULE-DECLARE-EXPRESSIONS",
        )):
            continue

        # Clean up description
        desc = (businessdescription or "No business description").strip()
        desc = " ".join(desc.split())  # remove extra whitespace
        if len(desc) > 400:
            desc = desc[:400] + "..."  # truncate overly long text

        # Recursive call for nested children
        calls = build_rule_hierarchy(key, visited)

        # Build the node as nested dict
        node = {
            label: {
                "Name": name,
                "Description": desc
            }
        }

        if calls:
            node[label]["Calls"] = calls

        hierarchy.append(node)

    return hierarchy

# -----------------------------
# Build hierarchy from top-level case types
# -----------------------------
cursor.execute("""
    SELECT DISTINCT top_rule_key
    FROM rule_references
    WHERE top_rule_key LIKE 'RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT'
""")
top_keys = [row[0] for row in cursor.fetchall()]

final_output = []

for key in top_keys:
    children = build_rule_hierarchy(key)
    case_type = {
        "CaseType": {
            "Name": key,
            "Calls": children
        }
    }
    final_output.append(case_type)

# -----------------------------
# Output JSON
# -----------------------------
json_output = json.dumps(final_output, indent=2, ensure_ascii=False)
print(json_output)

# Optionally save to file
with open("rule_hierarchy_with_desc.json", "w", encoding="utf-8") as f:
    f.write(json_output)

conn.close()


[
  {
    "CaseType": {
      "Name": "RULE-OBJ-CASETYPE UNILEVER-SEAA-MMDM-WORK-PRODUCTMANAGEMENT-MAINT PYDEFAULT #20240208T100011.667 GMT",
      "Calls": [
        {
          "Flow": {
            "Name": "Cancellation",
            "Description": "This rule, named \"NewSample\", starts when a new case is created. It then moves to two utility activities, \"InvokeCancelCase\" and \"DeleteInstinstancesForCancelCase\". The \"InvokeCancelCase\" activity invokes a cancel case with a specific page class, while the \"DeleteInstinstancesForCancelCase\" activity deletes instances for a cancel case with a specific case ID. Finally, the rule ends.",
            "Calls": [
              {
                "Model": {
                  "Name": "RemoveProjectTeam",
                  "Description": "This rule cancels an initiative when its associated case is closed. It iterates through the project team members, setting their assignment keys to empty strings. The rule also removes the SLA end date. 

In [ ]:
# DEBUG - Add initial Records (print SQL queries instead of executing)
import json
import os
import sqlite3

# Folder containing JSON files
folder_path = '/content/allrulessample'

# SQLite database file
db_file = 'rules_new.db'

# Connect to SQLite (optional, no writes will occur)
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Process all JSON files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extract top-level rule info
        top_rule_info = (
            data.get("pzInsKey", ""),
            data.get("pxObjClass", ""),
            data.get("pyClassName", ""),
            data.get("pyRuleName", ""),
            data.get("pyDescription", "")
        )

        # Print top rule insert query
        print("\n📘 Would execute:")
        print("INSERT INTO top_rules (key, rule_type, rule_class, name, description)")
        print("VALUES (?, ?, ?, ?, ?)")
        print("→ Params:", top_rule_info)

        top_rule_key = data.get("pzInsKey", "")

        # Extract embedded rule references
        for ref in data.get("pxRuleReferences", []):
            rule_info = (
                top_rule_key,
                ref.get("pxRuleObjClass", ""),
                ref.get("pxRuleClassName", ""),
                ref.get("pyRuleName", "")
            )

            # Print rule reference insert query
            print("\n   🔗 Would execute:")
            print("   INSERT INTO rule_references (top_rule_key, rule_type, rule_class, name)")
            print("   VALUES (?, ?, ?, ?)")
            print("   → Params:", rule_info)

# Close connection (no writes)
conn.close()

print("\n✅ All JSON files processed — SQL queries printed only (no inserts executed).")


Streaming output truncated to the last 5000 lines.
   INSERT INTO rule_references (top_rule_key, rule_type, rule_class, name)
   VALUES (?, ?, ?, ?)
   → Params: ('RULE-HTML-HARNESS DATA-PORTAL PULSE #20210506T204026.437 GMT', 'Rule-Obj-When', 'Data-Portal', 'recordEvent')

   🔗 Would execute:
   INSERT INTO rule_references (top_rule_key, rule_type, rule_class, name)
   VALUES (?, ?, ?, ?)
   → Params: ('RULE-HTML-HARNESS DATA-PORTAL PULSE #20210506T204026.437 GMT', 'Rule-HTML-Section', 'Data-Portal', 'PulseHeaderWrapper')

   🔗 Would execute:
   INSERT INTO rule_references (top_rule_key, rule_type, rule_class, name)
   VALUES (?, ?, ?, ?)
   → Params: ('RULE-HTML-HARNESS DATA-PORTAL PULSE #20210506T204026.437 GMT', 'Rule-HTML-Fragment', '', 'pzCompactPDFStyles')

📘 Would execute:
INSERT INTO top_rules (key, rule_type, rule_class, name, description)
VALUES (?, ?, ?, ?, ?)
→ Params: ('RULE-UTILITY-FUNCTION UNILEVERUTILS GCDOFTWODOUBLE--(DOUBLE,DOUBLE) #20210326T143435.451 GMT', 'Rule-Ut